# VacationPy

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mitsamiouli,-11.3847,43.2844,23.16,88,20,2.57,KM,1695353674
1,1,vila velha,-20.3297,-40.2925,21.64,91,20,2.06,BR,1695353675
2,2,juneau,58.3019,-134.4197,12.00,100,100,2.57,US,1695353483
3,3,kabalo,-6.0500,26.9167,22.98,72,100,0.89,CD,1695353675
4,4,shumikha,55.2261,63.2961,14.85,66,96,6.13,RU,1695353676


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
"Lng",
"Lat",
geo=True,
tiles= "OSM",
frame_width = 600,
frame_height = 600,
scale = 1,
size = "Humidity",
color = "City"
).opts(title="Humidity (%) of Cities across the World")

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_df = city_data_df.dropna()

# Drop any rows with null values
cleaned_df = city_df.loc[(city_df["Max Temp"] > 21) & (city_df["Max Temp"] < 27) & (city_df["Wind Speed"]<4.5) & (city_df["Cloudiness"] <50), :]

# Display sample data
cleaned_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mitsamiouli,-11.3847,43.2844,23.16,88,20,2.57,KM,1695353674
1,1,vila velha,-20.3297,-40.2925,21.64,91,20,2.06,BR,1695353675
23,23,souran,35.2916,36.7488,23.16,62,0,1.58,SY,1695353680
32,32,newman,37.3138,-121.0208,23.13,63,6,2.24,US,1695353683
44,44,sao vicente,-23.9631,-46.3919,25.15,82,0,1.34,BR,1695353686


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df.copy()
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mitsamiouli,-11.3847,43.2844,23.16,88,20,2.57,KM,1695353674
1,1,vila velha,-20.3297,-40.2925,21.64,91,20,2.06,BR,1695353675
23,23,souran,35.2916,36.7488,23.16,62,0,1.58,SY,1695353680
32,32,newman,37.3138,-121.0208,23.13,63,6,2.24,US,1695353683
44,44,sao vicente,-23.9631,-46.3919,25.15,82,0,1.34,BR,1695353686


In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mitsamiouli,KM,-11.3847,43.2844,88,
1,vila velha,BR,-20.3297,-40.2925,91,
23,souran,SY,35.2916,36.7488,62,
32,newman,US,37.3138,-121.0208,63,
44,sao vicente,BR,-23.9631,-46.3919,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,  
    "format": "json",
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):

        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mitsamiouli - nearest hotel: Hotel le Paradis
vila velha - nearest hotel: Hotel Prainha
souran - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
lexington - nearest hotel: Extended Stay America
sao gabriel da cachoeira - nearest hotel: Hotel Deus Me Deu
mahanoro - nearest hotel: Vanilla Bungalow
al fulah - nearest hotel: No hotel found
novo aripuana - nearest hotel: Hotel Novo Aripuanã
san carlos - nearest hotel: Hotel Villas del Norte
ariquemes - nearest hotel: Aquarius
siavonga - nearest hotel: Lake Safari Lodge Annex
miracema do tocantins - nearest hotel: No hotel found
valle vista - nearest hotel: Quality Inn Hemet - San Jacinto
salinopolis - nearest hotel: Hotel Salinópolis
at taj - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
antsiranana - nearest hotel: Central
zyrya - nearest hotel: No hotel found
aghajari - nearest hotel: No hotel found
sa

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mitsamiouli,KM,-11.3847,43.2844,88,Hotel le Paradis
1,vila velha,BR,-20.3297,-40.2925,91,Hotel Prainha
23,souran,SY,35.2916,36.7488,62,No hotel found
32,newman,US,37.3138,-121.0208,63,No hotel found
44,sao vicente,BR,-23.9631,-46.3919,82,Pousada Vitória
51,lexington,US,37.9887,-84.4777,65,Extended Stay America
55,sao gabriel da cachoeira,BR,-0.1303,-67.0892,89,Hotel Deus Me Deu
101,mahanoro,MG,-19.9000,48.8000,74,Vanilla Bungalow
126,al fulah,SD,11.7329,28.3579,68,No hotel found
139,novo aripuana,BR,-5.1206,-60.3797,61,Hotel Novo Aripuanã


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    frame_width = 600,
    frame_height = 600, 
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot 



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)